In [3]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [4]:
sample_submission = pd.read_csv('data/SampleSubmission.csv')
test_customer = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')
vendors = pd.read_csv('data/vendors.csv')

In [5]:
print(sample_submission.head())
print(len(sample_submission))

# need to generate test dataset with same number of rows and combined column : CID, LOC_NUM, VENDOR
# (akeed_customer_id, location_number, id(in vendor.csv))

  CID X LOC_NUM X VENDOR  target
0      Z59FTQD X 0 X 243       0
1      0JP29SK X 0 X 243       0
2      0JP29SK X 1 X 243       0
3      0JP29SK X 2 X 243       0
4      0JP29SK X 3 X 243       0
1672000


In [6]:
# drop duplicates for test customers
test_customer.drop_duplicates(subset = "akeed_customer_id", keep = False, inplace = True)
# merge test customer and locations
test = test_customer.merge(test_locations, left_on = "akeed_customer_id", right_on = "customer_id")

In [7]:
# add all rows of vendor to each test row
test_ = test.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

test_['akeed_customer_id'] = test_['akeed_customer_id'].astype(str)
test_['location_number'] = test_['location_number'].astype(str)
test_['id'] = test_['id'].astype(str)

# generate new column 'CID X LOC_NUM X VENDOR' by combining the 3 columns
test_['CID X LOC_NUM X VENDOR'] = test_["akeed_customer_id"]+" X "+ \
test_["location_number"] +" X "+ test_["id"]

In [8]:
# some of the rows in sample_submission are not present in the generated test file as these rows contain 
# akeed_customer_id's never seen before in test_customer file
len(test_), len(sample_submission)

(1630600, 1672000)

In [9]:
#generate new dataframe containing the missing rows
a,b = test_['CID X LOC_NUM X VENDOR'], sample_submission['CID X LOC_NUM X VENDOR']
a,b = pd.DataFrame(a), pd.DataFrame(b)
ds1 = set([tuple(line) for line in a.values])
ds2 = set([tuple(line) for line in b.values])
missing = pd.DataFrame(list(ds2.difference(ds1)))

missing.head()

,0
0,VX18JF5 X 0 X 55
1,H8R3FMH X 0 X 274
2,QF0F8SO X 1 X 846
3,3F69SKX X 0 X 681
4,VX18JF5 X 1 X 845


In [10]:
# null count for columns
for column in list(test_.columns):
    print(column, " : ", test_[column].isna().sum())

akeed_customer_id  :  0
gender  :  479700
dob  :  1488100
status_x  :  0
verified_x  :  0
language_x  :  435800
created_at_x  :  0
updated_at_x  :  0
customer_id  :  0
location_number  :  0
location_type  :  745800
latitude_x  :  300
longitude_x  :  300
id  :  0
authentication_id  :  0
latitude_y  :  0
longitude_y  :  0
vendor_category_en  :  0
vendor_category_id  :  0
delivery_charge  :  0
serving_distance  :  0
is_open  :  0
OpeningTime  :  146754
OpeningTime2  :  146754
prepration_time  :  0
commission  :  244590
is_akeed_delivering  :  0
discount_percentage  :  0
status_y  :  0
verified_y  :  0
rank  :  0
language_y  :  244590
vendor_rating  :  0
sunday_from_time1  :  16306
sunday_to_time1  :  16306
sunday_from_time2  :  945748
sunday_to_time2  :  945748
monday_from_time1  :  0
monday_to_time1  :  0
monday_from_time2  :  945748
monday_to_time2  :  945748
tuesday_from_time1  :  16306
tuesday_to_time1  :  16306
tuesday_from_time2  :  962054
tuesday_to_time2  :  962054
wednesday_from_

In [11]:
# unique values for each column
for column in list(test_.columns):
    print(column, " : ", len(test_[column].unique()))

akeed_customer_id  :  9743
gender  :  7
dob  :  53
status_x  :  2
verified_x  :  2
language_x  :  2
created_at_x  :  9636
updated_at_x  :  8896
customer_id  :  9743
location_number  :  12
location_type  :  4
latitude_x  :  16198
longitude_x  :  16208
id  :  100
authentication_id  :  100
latitude_y  :  99
longitude_y  :  100
vendor_category_en  :  2
vendor_category_id  :  2
delivery_charge  :  2
serving_distance  :  10
is_open  :  2
OpeningTime  :  69
OpeningTime2  :  12
prepration_time  :  14
commission  :  2
is_akeed_delivering  :  1
discount_percentage  :  5
status_y  :  2
verified_y  :  2
rank  :  2
language_y  :  2
vendor_rating  :  12
sunday_from_time1  :  20
sunday_to_time1  :  28
sunday_from_time2  :  17
sunday_to_time2  :  8
monday_from_time1  :  20
monday_to_time1  :  28
monday_from_time2  :  17
monday_to_time2  :  8
tuesday_from_time1  :  21
tuesday_to_time1  :  27
tuesday_from_time2  :  16
tuesday_to_time2  :  7
wednesday_from_time1  :  19
wednesday_to_time1  :  28
wednesday

In [12]:
test_.drop(['gender', 'dob', 'language_x', 'location_type', 'vendor_tag_name','is_akeed_delivering', \
           'country_id', 'city_id', 'display_orders', 'one_click_vendor', 'open_close_flags'], axis=1, inplace=True)

In [13]:
missing

,0
0,VX18JF5 X 0 X 55
1,H8R3FMH X 0 X 274
2,QF0F8SO X 1 X 846
3,3F69SKX X 0 X 681
4,VX18JF5 X 1 X 845
...,...
41395,T48O0RF X 0 X 161
41396,WXIB913 X 0 X 299
41397,71F29FK X 0 X 849
41398,FOTZ1VY X 0 X 299


In [14]:
# splitting the column into 3 columns - CID, LOC_NUM, VENDOR

missing[0] = missing[0].astype(str)
missing['akeed_customer_id'], missing['location_number'], missing['id'] = np.nan, np.nan, np.nan
for i in tqdm(range(len(missing))):
    missing.loc[i,'akeed_customer_id'], missing.loc[i,'location_number'], missing.loc[i,'id'] = \
    missing.loc[i,0].split(' X ')

In [15]:
missing

,0,akeed_customer_id,location_number,id
0,VX18JF5 X 0 X 55,VX18JF5,0,55
1,H8R3FMH X 0 X 274,H8R3FMH,0,274
2,QF0F8SO X 1 X 846,QF0F8SO,1,846
3,3F69SKX X 0 X 681,3F69SKX,0,681
4,VX18JF5 X 1 X 845,VX18JF5,1,845
...,...,...,...,...
41395,T48O0RF X 0 X 161,T48O0RF,0,161
41396,WXIB913 X 0 X 299,WXIB913,0,299
41397,71F29FK X 0 X 849,71F29FK,0,849
41398,FOTZ1VY X 0 X 299,FOTZ1VY,0,299


In [16]:
# merging missing dataframe with the vendors dataframe
missing['id'] = missing['id'].astype(int)
missing_ = missing.merge(vendors, left_on = "id", right_on = "id")
missing_ = missing_.rename(columns={0: 'CID X LOC_NUM X VENDOR'})

missing_.head()

,CID X LOC_NUM X VENDOR,akeed_customer_id,location_number,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,VX18JF5 X 0 X 55,VX18JF5,0,55,118651.0,-1.170154,0.103477,Restaurants,2.0,0.7,...,1.0,"4,8,22,32,24,16,28","Breakfast,Desserts,Grills,Milkshakes,Salads,Sa...",Y,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1
1,QCU280K X 5 X 55,QCU280K,5,55,118651.0,-1.170154,0.103477,Restaurants,2.0,0.7,...,1.0,"4,8,22,32,24,16,28","Breakfast,Desserts,Grills,Milkshakes,Salads,Sa...",Y,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1
2,79KP2HO X 0 X 55,79KP2HO,0,55,118651.0,-1.170154,0.103477,Restaurants,2.0,0.7,...,1.0,"4,8,22,32,24,16,28","Breakfast,Desserts,Grills,Milkshakes,Salads,Sa...",Y,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1
3,QYMTNJO X 0 X 55,QYMTNJO,0,55,118651.0,-1.170154,0.103477,Restaurants,2.0,0.7,...,1.0,"4,8,22,32,24,16,28","Breakfast,Desserts,Grills,Milkshakes,Salads,Sa...",Y,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1
4,1YAP71M X 1 X 55,1YAP71M,1,55,118651.0,-1.170154,0.103477,Restaurants,2.0,0.7,...,1.0,"4,8,22,32,24,16,28","Breakfast,Desserts,Grills,Milkshakes,Salads,Sa...",Y,1.0,1.0,2018-07-22 13:18:35,2020-04-02 19:49:47,3,1


In [17]:
extra_columns = [c for c in list(missing_.columns) if c not in list(test_.columns)]
for c in extra_columns:
    missing_.drop([c], axis=1, inplace=True)

In [18]:
len(missing_.columns), len(test_.columns)

(48, 62)

In [19]:
# combining missing_ dataframe with the test_ file to produce dataframe same size of the result
test2 = test_.append(missing_, ignore_index=True)
len(test2) == len(sample_submission)

True

In [20]:
extra = [a for a in test_.columns if a not in missing_.columns]
extra

['status_x',
 'verified_x',
 'created_at_x',
 'updated_at_x',
 'customer_id',
 'latitude_x',
 'longitude_x',
 'latitude_y',
 'longitude_y',
 'status_y',
 'verified_y',
 'language_y',
 'created_at_y',
 'updated_at_y']